<a href="https://colab.research.google.com/github/HimashreeLingaraju/AI-basedDocumentSummaryGenerator/blob/main/nlm_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install required packages
!pip install nltk spacy scikit-learn transformers gensim PyPDF2 python-docx python-pptx openpyxl opencv-python pytesseract pandas
!pip install pytesseract

# Download spaCy model
!python -m spacy download en_core_web_sm

# Download NLTK data
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

print("Setup complete!")

  Using cached https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.8.0/en_core_web_sm-3.8.0-py3-none-any.whl (12.8 MB)
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


AttributeError: module 'numpy' has no attribute '_no_nep50_warning'

In [ ]:
# Install required packages
!pip install numpy scipy nltk spacy scikit-learn transformers gensim PyPDF2 python-docx python-pptx openpyxl opencv-python pytesseract pandas
!pip install pytesseract

# Download spaCy model
!python -m spacy download en_core_web_sm

# Download NLTK data
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

print("Setup complete!")

  Using cached https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.8.0/en_core_web_sm-3.8.0-py3-none-any.whl (12.8 MB)
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


AttributeError: module 'numpy' has no attribute '_no_nep50_warning'

In [ ]:
!pip install numpy==1.24.4 --force-reinstall


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 111.7 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4


In [ ]:
# First, restart runtime and install dependencies in correct order
import sys
import subprocess
import importlib

def install_dependencies():
    """Install dependencies in the correct order to avoid conflicts."""

    # Uninstall conflicting packages first
    subprocess.run([sys.executable, "-m", "pip", "uninstall", "-y", "torch", "torchvision", "torchaudio"],
                   capture_output=True)

    # Install PyTorch with CPU support first
    subprocess.run([sys.executable, "-m", "pip", "install", "torch", "torchvision", "torchaudio",
                   "--index-url", "https://download.pytorch.org/whl/cpu"], check=True)

    # Install other dependencies
    packages = [
        "PyPDF2", "python-docx", "python-pptx", "openpyxl",
        "opencv-python-headless",  # Use headless version for Colab
        "pytesseract", "scikit-learn", "nltk", "spacy",
        "gensim", "pandas", "numpy"
    ]

    for package in packages:
        subprocess.run([sys.executable, "-m", "pip", "install", package], check=True)

    # Install transformers last
    subprocess.run([sys.executable, "-m", "pip", "install", "transformers"], check=True)

    # Install tesseract
    subprocess.run(["apt-get", "update"], check=True)
    subprocess.run(["apt-get", "install", "-y", "tesseract-ocr"], check=True)

    # Download spaCy model
    subprocess.run([sys.executable, "-m", "spacy", "download", "en_core_web_sm"], check=True)

# Install dependencies
print("Installing dependencies... This may take a few minutes.")
install_dependencies()
print("Dependencies installed successfully!")

# Now import everything
import logging
from pathlib import Path
from collections import defaultdict
from datetime import datetime
import PyPDF2
from docx import Document
from pptx import Presentation
from openpyxl import load_workbook
import cv2
import pytesseract
import re
import pandas as pd
from typing import Optional, Tuple, List, Dict
import nltk
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from gensim import corpora
from gensim.models import LdaModel
from collections import Counter
import numpy as np
import os

# Download NLTK data
print("Downloading NLTK data...")
nltk.download('punkt', quiet=True)
nltk.download('stopwords', quiet=True)
nltk.download('wordnet', quiet=True)
nltk.download('omw-1.4', quiet=True)

# Import transformers after everything else is set up
try:
    from transformers import pipeline
    TRANSFORMERS_AVAILABLE = True
    print("Transformers loaded successfully!")
except Exception as e:
    print(f"Warning: Transformers not available: {e}")
    TRANSFORMERS_AVAILABLE = False

class TextAnalyzer:
    def __init__(self):
        """Initialize the text analyzer with advanced NLP models."""
        try:
            # Load spaCy model
            self.nlp = spacy.load('en_core_web_sm')
            print("spaCy model loaded successfully!")
        except OSError:
            print("spaCy model not found. Downloading...")
            subprocess.run([sys.executable, "-m", "spacy", "download", "en_core_web_sm"], check=True)
            self.nlp = spacy.load('en_core_web_sm')

        self.lemmatizer = WordNetLemmatizer()
        self.stop_words = set(stopwords.words('english'))
        self.vectorizer = TfidfVectorizer(max_features=1000, stop_words='english', ngram_range=(1, 3))

        # Add BERT-based summarization pipeline with error handling
        self.summarizer = None
        if TRANSFORMERS_AVAILABLE:
            try:
                self.summarizer = pipeline("summarization", model="sshleifer/distilbart-cnn-12-6")
                print("Summarization model loaded successfully!")
            except Exception as e:
                print(f"Warning: Could not load summarization model: {e}")
                self.summarizer = None

        # For LDA topic modeling
        self.min_df = 2

    def preprocess_text(self, text: str) -> str:
        """Clean and preprocess text with enhanced techniques."""
        if not text:
            return ""

        text = re.sub(r'\s+', ' ', text.strip())
        text = re.sub(r'[^\w\s.]', '', text)

        try:
            doc = self.nlp(text[:1000])  # Limit text length for performance
            cleaned_tokens = [
                token.lemma_.lower() for token in doc
                if not token.is_stop and not token.is_punct and len(token.text) > 2
            ]
            return ' '.join(cleaned_tokens)
        except Exception as e:
            print(f"Error in text preprocessing: {e}")
            return text

    def extract_key_phrases(self, text: str, min_length: int = 2) -> List[str]:
        """Extract key phrases using spaCy."""
        if not text:
            return []

        try:
            doc = self.nlp(text[:1000])  # Limit for performance
            key_phrases = set()

            # Noun chunks and named entities
            for chunk in doc.noun_chunks:
                if len(chunk.text.split()) >= min_length:
                    key_phrases.add(chunk.text.strip())
            for ent in doc.ents:
                if len(ent.text.split()) >= min_length:
                    key_phrases.add(ent.text.strip())

            return sorted(list(key_phrases), key=len, reverse=True)[:10]
        except Exception as e:
            print(f"Error in key phrase extraction: {e}")
            return []

    def generate_smart_summary(self, text: str, target_length: int = 3) -> str:
        """Generate summary using available methods."""
        if not text or len(text.strip()) < 50:
            return text

        try:
            # Use BERT summarizer if available and text is long enough
            if self.summarizer and len(text) > 200:
                # Truncate text if too long for the model
                max_input_length = 1024
                if len(text) > max_input_length:
                    text = text[:max_input_length]

                summary = self.summarizer(
                    text,
                    max_length=100,
                    min_length=30,
                    do_sample=False
                )[0]['summary_text']
                return summary

            # Fallback to sentence extraction
            sentences = sent_tokenize(text)
            if len(sentences) <= target_length:
                return text

            if len(sentences) > 0:
                # Simple approach: take first, middle, and last sentences
                if len(sentences) >= 3:
                    indices = [0, len(sentences)//2, len(sentences)-1]
                    return ' '.join(sentences[i] for i in indices[:target_length])
                else:
                    return ' '.join(sentences[:target_length])

            return text[:500] + '...'

        except Exception as e:
            print(f"Error in summary generation: {str(e)}")
            return text[:500] + '...'

    def extract_main_topics(self, text: str, num_topics: int = 5) -> List[str]:
        """Extract topics using simple frequency analysis."""
        if not text:
            return []

        try:
            # Simple frequency-based topic extraction
            doc = self.nlp(text[:1000])

            # Extract meaningful words (nouns, adjectives, proper nouns)
            meaningful_words = [
                token.lemma_.lower() for token in doc
                if (token.pos_ in ['NOUN', 'ADJ', 'PROPN'] and
                    not token.is_stop and
                    not token.is_punct and
                    len(token.text) > 2)
            ]

            if not meaningful_words:
                return []

            # Get most common words as topics
            word_freq = Counter(meaningful_words)
            topics = [word for word, _ in word_freq.most_common(num_topics)]

            return topics

        except Exception as e:
            print(f"Error in topic extraction: {str(e)}")
            return []

    def extract_key_points(self, text: str, min_words: int = 5) -> List[str]:
        """Extract key points with semantic importance."""
        if not text:
            return []

        try:
            sentences = sent_tokenize(text)
            key_points = []

            for sent in sentences:
                if len(sent.split()) >= min_words:
                    # Simple scoring based on sentence length and content
                    doc = self.nlp(sent)
                    score = (
                        len(doc.ents) * 2 +  # Weight named entities higher
                        len([t for t in doc if t.pos_ in ['NOUN', 'VERB']]) +
                        len([t for t in doc if t.like_num]) * 1.5  # Weight numbers
                    )
                    if score > 0:
                        key_points.append((sent.strip(), score))

            # Sort by score and return top 5
            key_points.sort(key=lambda x: x[1], reverse=True)
            return [point[0] for point in key_points[:5]]

        except Exception as e:
            print(f"Error in key points extraction: {str(e)}")
            return []

class DocumentProcessor:
    def __init__(self, base_dir: str = "/content/input_dir", output_dir: str = "/content/output_dir"):
        self.base_dir = Path(base_dir)
        self.output_dir = Path(output_dir)
        self.output_dir.mkdir(parents=True, exist_ok=True)
        self.base_dir.mkdir(parents=True, exist_ok=True)
        self.index = defaultdict(list)
        self.summaries = {}
        self.text_analyzer = TextAnalyzer()

        # Initialize sentiment analyzer with error handling
        self.sentiment_analyzer = None
        if TRANSFORMERS_AVAILABLE:
            try:
                self.sentiment_analyzer = pipeline("sentiment-analysis", model="cardiffnlp/twitter-roberta-base-sentiment-latest")
                print("Sentiment analyzer loaded successfully!")
            except Exception as e:
                print(f"Warning: Could not load sentiment analyzer: {e}")
                self.sentiment_analyzer = None

        self.setup_logging()

    def setup_logging(self) -> None:
        """Configure logging for the document processor."""
        log_file = self.output_dir / f"processing_{datetime.now().strftime('%Y%m%d_%H%M%S')}.log"
        logging.basicConfig(
            level=logging.INFO,
            format='%(asctime)s - %(levelname)s - %(message)s',
            handlers=[
                logging.FileHandler(log_file),
                logging.StreamHandler()
            ]
        )

    def scan_directory(self) -> dict[str, list[str]]:
        """Scans directory and categorizes files by type."""
        file_types = defaultdict(list)
        logging.info(f"Scanning directory: {self.base_dir}")

        for file_path in self.base_dir.rglob('*'):
            if file_path.is_file():
                ext = file_path.suffix.lower()
                file_types[ext].append(str(file_path))

        logging.info(f"Found {sum(len(files) for files in file_types.values())} files")
        return dict(file_types)

    def create_file_summary(self, file_path: str) -> dict:
        """Creates detailed file summary."""
        path = Path(file_path)
        stats = path.stat()

        summary = {
            'filename': path.name,
            'path': str(path),
            'size': stats.st_size,
            'size_mb': round(stats.st_size / (1024 * 1024), 2),
            'modified': datetime.fromtimestamp(stats.st_mtime).isoformat(),
            'created': datetime.fromtimestamp(stats.st_ctime).isoformat(),
            'type': path.suffix.lower(),
            'pages': self.get_page_count(path),
            'has_images': False,
            'has_tables': False,
            'has_text': False,
            'content_summary': '',
            'key_points': [],
            'main_topics': [],
            'sentiment': 'N/A',
            'entities': []
        }
        return summary

    def get_page_count(self, path: Path) -> int:
        """Get page count for various document types."""
        try:
            ext = path.suffix.lower()
            if ext == '.pdf':
                with open(path, 'rb') as f:
                    return len(PyPDF2.PdfReader(f).pages)
            elif ext == '.docx':
                doc = Document(path)
                return len([p for p in doc.paragraphs if p.text.strip()])
            elif ext == '.pptx':
                prs = Presentation(path)
                return len(prs.slides)
            elif ext == '.xlsx':
                wb = load_workbook(path, read_only=True)
                return len(wb.sheetnames)
        except Exception as e:
            logging.error(f"Error getting page count for {path}: {str(e)}")
        return 1

    def extract_text_from_image(self, image_path: str) -> str:
        """Extract text from images using OCR."""
        try:
            image = cv2.imread(image_path)
            if image is None:
                return ""

            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            text = pytesseract.image_to_string(gray)
            return text.strip()
        except Exception as e:
            logging.error(f"Error performing OCR on {image_path}: {str(e)}")
            return ""

    def extract_text_from_pdf(self, pdf_path: str) -> Tuple[str, bool, bool]:
        """Extract text, check for images and tables in PDF."""
        try:
            text = []
            has_images = False
            has_tables = False

            with open(pdf_path, 'rb') as f:
                pdf = PyPDF2.PdfReader(f)
                for page in pdf.pages:
                    page_text = page.extract_text()
                    text.append(page_text)

                    # Simple heuristics for detecting images and tables
                    if 'image' in page_text.lower() or 'figure' in page_text.lower():
                        has_images = True

                    if re.search(r'[\t|]{2,}', page_text) or re.search(r'(\s{3,}[\w\d]+){3,}', page_text):
                        has_tables = True

            return '\n'.join(text), has_images, has_tables
        except Exception as e:
            logging.error(f"Error processing PDF {pdf_path}: {str(e)}")
            return "", False, False

    def extract_text_from_word(self, docx_path: str) -> Tuple[str, bool, bool]:
        """Extract text and check for images and tables in Word document."""
        try:
            doc = Document(docx_path)
            text = []
            has_images = False
            has_tables = len(doc.tables) > 0

            for paragraph in doc.paragraphs:
                if paragraph.text.strip():
                    text.append(paragraph.text)

            # Simple check for images
            for paragraph in doc.paragraphs:
                for run in paragraph.runs:
                    if hasattr(run, '_element') and run._element.findall('.//pic:pic'):
                        has_images = True
                        break

            return '\n'.join(text), has_images, has_tables
        except Exception as e:
            logging.error(f"Error processing Word document {docx_path}: {str(e)}")
            return "", False, False

    def extract_text_from_excel(self, xlsx_path: str) -> Tuple[str, bool]:
        """Extract text and check for tables in Excel workbook."""
        try:
            df_list = pd.read_excel(xlsx_path, sheet_name=None)
            text = []
            has_tables = False

            for sheet_name, df in df_list.items():
                if not df.empty:
                    has_tables = True
                    text.append(f"Sheet: {sheet_name}")
                    # Convert to string but limit size
                    sheet_text = df.head(10).to_string()  # Only first 10 rows
                    text.append(sheet_text)
                    text.append("\n")

            return '\n'.join(text), has_tables
        except Exception as e:
            logging.error(f"Error processing Excel file {xlsx_path}: {str(e)}")
            return "", False

    def extract_text_from_powerpoint(self, pptx_path: str) -> Tuple[str, bool, bool]:
        """Extract text and check for images and tables in PowerPoint presentation."""
        try:
            prs = Presentation(pptx_path)
            text = []
            has_images = False
            has_tables = False

            for slide_num, slide in enumerate(prs.slides):
                text.append(f"--- Slide {slide_num + 1} ---")
                for shape in slide.shapes:
                    if hasattr(shape, 'text') and shape.text.strip():
                        text.append(shape.text)

                    # Check for images and tables
                    if hasattr(shape, 'shape_type'):
                        if shape.shape_type == 13:  # Picture
                            has_images = True
                        elif shape.shape_type == 19:  # Table
                            has_tables = True

            return '\n'.join(text), has_images, has_tables
        except Exception as e:
            logging.error(f"Error processing PowerPoint file {pptx_path}: {str(e)}")
            return "", False, False

    def extract_text_content(self, file_path: str) -> Tuple[str, bool, bool]:
        """Extract text content from supported file types."""
        path = Path(file_path)
        ext = path.suffix.lower()

        try:
            if ext in ['.jpg', '.jpeg', '.png', '.bmp', '.tiff']:
                text = self.extract_text_from_image(str(path))
                return text, True, False
            elif ext == '.pdf':
                return self.extract_text_from_pdf(str(path))
            elif ext == '.docx':
                return self.extract_text_from_word(str(path))
            elif ext == '.xlsx':
                text, has_tables = self.extract_text_from_excel(str(path))
                return text, False, has_tables
            elif ext == '.pptx':
                return self.extract_text_from_powerpoint(str(path))
            elif ext == '.txt':
                with open(path, 'r', encoding='utf-8', errors='ignore') as f:
                    return f.read(), False, False
        except Exception as e:
            logging.error(f"Error extracting text from {path}: {str(e)}")

        return "", False, False

    def generate_content_summary(self, text: str) -> Tuple[str, list[str], list[str], str, list[str]]:
        """Enhanced content analysis with sentiment and entities."""
        if not text or len(text.strip()) < 50:
            return "", [], [], "N/A", []

        # Limit text length for processing
        text = text[:2000]

        summary = self.text_analyzer.generate_smart_summary(text, 3)
        key_points = self.text_analyzer.extract_key_points(text)
        main_topics = self.text_analyzer.extract_main_topics(text)

        # Sentiment analysis
        sentiment = "N/A"
        if self.sentiment_analyzer:
            try:
                # Limit text for sentiment analysis
                sentiment_text = text[:512]
                result = self.sentiment_analyzer(sentiment_text)[0]
                sentiment = result['label']
            except Exception as e:
                print(f"Error in sentiment analysis: {e}")

        # Entity extraction
        entities = []
        try:
            doc = self.text_analyzer.nlp(text[:1000])
            entities = list(set(ent.text for ent in doc.ents))[:10]
        except Exception as e:
            print(f"Error in entity extraction: {e}")

        return summary, key_points, main_topics, sentiment, entities

    def index_keywords(self, content: str, file_path: str) -> None:
        """Creates keyword index for searchability."""
        words = re.findall(r'\w+', content.lower())
        for word in words:
            if len(word) > 2:
                self.index[word].append(file_path)

    def search_keyword(self, keyword: str) -> list[str]:
        """Searches for files containing the keyword."""
        return self.index.get(keyword.lower(), [])

    def format_summary_text(self) -> str:
        """Format the summary with sentiment and entities."""
        lines = ["DOCUMENT PROCESSING SUMMARY REPORT", "=" * 50, ""]

        files_by_type = defaultdict(list)
        for file_path, summary in self.summaries.items():
            files_by_type[summary['type']].append(summary)

        lines.extend([
            "OVERALL STATISTICS",
            "-" * 20,
            f"Total files processed: {len(self.summaries)}",
            f"File types found: {', '.join(files_by_type.keys())}",
            f"Processing date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}",
            ""
        ])

        for file_type, files in files_by_type.items():
            lines.extend([
                f"\nFile type: {file_type}",
                f"Number of files: {len(files)}",
                f"Total size: {sum(f['size_mb'] for f in files):.2f} MB",
                "Files:"
            ])
            for file_summary in files:
                lines.extend([
                    f"\n  Filename: {file_summary['filename']}",
                    f"  Size: {file_summary['size_mb']:.2f} MB",
                    f"  Pages: {file_summary['pages']}",
                    f"  Contains images: {'Yes' if file_summary['has_images'] else 'No'}",
                    f"  Contains tables: {'Yes' if file_summary['has_tables'] else 'No'}",
                    f"  Contains text: {'Yes' if file_summary['has_text'] else 'No'}",
                    f"  Last modified: {file_summary['modified']}",
                    f"  Sentiment: {file_summary['sentiment']}",
                    f"  Entities: {', '.join(file_summary['entities'][:5]) or 'None'}"
                ])

                if file_summary['content_summary']:
                    lines.extend([
                        "\n  CONTENT ANALYSIS",
                        "  " + "-" * 18,
                        f"  Summary: {file_summary['content_summary']}",
                        "\n  Key Points:"
                    ])
                    for point in file_summary['key_points']:
                        lines.append(f"   - {point}")
                    lines.append("\n  Main Topics:")
                    for topic in file_summary['main_topics']:
                        lines.append(f"   - {topic}")
                    lines.append("")

        keyword_freq = {k: len(v) for k, v in self.index.items()}
        top_keywords = sorted(keyword_freq.items(), key=lambda x: x[1], reverse=True)[:20]
        lines.extend([
            "\nKEYWORD INDEX SUMMARY",
            "-" * 20,
            f"Total unique keywords: {len(self.index)}",
            "\nMost common keywords (top 20):"
        ])
        for keyword, freq in top_keywords:
            lines.append(f"  {keyword}: {freq} occurrences")

        return "\n".join(lines)

    def export_summary(self) -> None:
        """Exports summary to text format."""
        if not self.summaries:
            return

        summary_text = self.format_summary_text()
        summary_path = self.output_dir / "document_summary.txt"
        with open(summary_path, 'w', encoding='utf-8') as f:
            f.write(summary_text)

        index_path = self.output_dir / "keyword_index.txt"
        with open(index_path, 'w', encoding='utf-8') as f:
            f.write("KEYWORD INDEX\n")
            f.write("=" * 50 + "\n\n")
            for keyword, files in sorted(list(self.index.items())[:100]):  # Limit output
                f.write(f"Keyword: {keyword}\n")
                f.write("Files:\n")
                for file_path in files:
                    f.write(f"  - {file_path}\n")
                f.write("\n")

        logging.info(f"Exported summary to {summary_path}")
        logging.info(f"Exported keyword index to {index_path}")

    def process_documents(self) -> None:
        """Main processing function with enhanced analysis."""
        start_time = datetime.now()
        logging.info("Starting document processing")

        file_types = self.scan_directory()
        total_files = sum(len(files) for files in file_types.values())

        if total_files == 0:
            logging.info("No files found to process")
            print("No files found in the input directory. Creating sample files...")
            self.create_sample_files()
            file_types = self.scan_directory()
            total_files = sum(len(files) for files in file_types.values())

        processed_files = 0

        for file_list in file_types.values():
            for file_path in file_list:
                try:
                    processed_files += 1
                    logging.info(f"Processing file {processed_files}/{total_files}: {file_path}")

                    summary = self.create_file_summary(file_path)
                    content, has_images, has_tables = self.extract_text_content(file_path)

                    if content:
                        summary['has_text'] = True
                        summary['has_images'] = has_images
                        summary['has_tables'] = has_tables
                        self.index_keywords(content, file_path)

                        content_summary, key_points, main_topics, sentiment, entities = self.generate_content_summary(content)
                        summary.update({
                            'content_summary': content_summary,
                            'key_points': key_points,
                            'main_topics': main_topics,
                            'sentiment': sentiment,
                            'entities': entities
                        })

                    self.summaries[file_path] = summary

                except Exception as e:
                    logging.error(f"Error processing {file_path}: {str(e)}")

        self.export_summary()
        end_time = datetime.now()
        duration = (end_time - start_time).total_seconds()
        logging.info(f"Processing completed in {duration:.2f} seconds")
        logging.info(f"Processed {processed_files} files")
        logging.info(f"Results exported to {self.output_dir}")

    def create_sample_files(self):
        """Create sample files for demonstration."""
        sample_texts = [
            {
                'filename': 'technology_report.txt',
                'content': '''
                Artificial Intelligence and Machine Learning Report

                This comprehensive report examines the current state of artificial intelligence
                and machine learning technologies in 2024. The document covers various aspects
                including natural language processing, computer vision, and deep learning algorithms.

                Key findings include:
                - AI adoption has increased by 300% in enterprise environments
                - Machine learning models are becoming more efficient and accurate
                - Natural language processing has reached human-level performance in many tasks
                - Computer vision applications are expanding into healthcare and autonomous vehicles

                The report concludes that AI technology will continue to transform industries
                and create new opportunities for innovation and growth.
                '''
            },
            {
                'filename': 'business_analysis.txt',
                'content': '''
                Quarterly Business Performance Analysis

                This document presents a detailed analysis of our company's performance
                during Q3 2024. The analysis includes financial metrics, market trends,
                and strategic recommendations for the upcoming quarter.

                Financial Highlights:
                - Revenue increased by 15% compared to Q2 2024
                - Operating expenses decreased by 8%
                - Net profit margin improved to 12.5%
                - Customer acquisition cost reduced by 20%

                Market Analysis:
                The technology sector showed strong growth with emerging trends in
                cloud computing, cybersecurity, and digital transformation driving
                demand for our services.

                Recommendations:
                1. Increase investment in R&D for emerging technologies
                2. Expand market presence in Asia-Pacific region
                3. Strengthen partnerships with key technology vendors
                '''
            }
        ]

        for sample in sample_texts:
            file_path = self.base_dir / sample['filename']
            with open(file_path, 'w', encoding='utf-8') as f:
                f.write(sample['content'])

        print(f"Created {len(sample_texts)} sample files in {self.base_dir}")

# Example usage for Google Colab
def demo_usage():
    """Demonstrate how to use the document processor in Colab."""

    print("Starting Document Processor Demo for Google Colab")
    print("="*60)

    # Initialize and run processor
    processor = DocumentProcessor()
    processor.process_documents()

    # Demonstrate search functionality
    print("\n" + "="*50)
    print("SEARCH DEMONSTRATION")
    print("="*50)

    search_terms = ["technology", "analysis", "artificial", "business"]
    for term in search_terms:
        results = processor.search_keyword(term)
        if results:
            print(f"\nFiles containing '{term}':")
            for file_path in results:
                print(f"- {Path(file_path).name}")
        else:
            print(f"\nNo files found containing '{term}'")

    # Display summary
    print(f"\nSummary files created in: {processor.output_dir}")
    print("Check the following files:")
    print("- document_summary.txt")
    print("- keyword_index.txt")
    print("- processing log file")

    # Show summary content
    summary_file = processor.output_dir / "document_summary.txt"
    if summary_file.exists():
        print("\n" + "="*50)
        print("SUMMARY PREVIEW")
        print("="*50)
        with open(summary_file, 'r', encoding='utf-8') as f:
            content = f.read()
            print(content[:1500] + "..." if len(content) > 1500 else content)

# Run the demo
if __name__ == "__main__":
    demo_usage()

Installing dependencies... This may take a few minutes.
Dependencies installed successfully!


AttributeError: module 'numpy' has no attribute '_no_nep50_warning'

In [ ]:
*!pip install --upgrade --force-reinstall numpy
!pip install --upgrade --force-reinstall scipy pandas matplotlib


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 74.8 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fastai 2.7.19 requires torch<2.7,>=1.10, which is not installed.
fastai 2.7.19 requires torchvision>=0.11, which is not installed.
accelerate 1.6.0 requires torch>=2.0.0, which is not installed.
sentence-transformers 4.1.0 requires torch>=1.11.0, which is not installed.
peft 0.15.2 requires torch>=1.13.0, which is not installed.
gensim 4.3.3 requires numpy<2.0,>=1.18.5, but you have numpy 2.2.6 which is incompatible.
tsfresh 0.21.0 requires scipy>=1.14.0; python_version >= "3.10", but you have scipy 1.13.1 whi